<a href="https://colab.research.google.com/github/afortuny/DeepLearningFastAI/blob/main/14_2_Resnet_Deep_Dive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>